In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

df = pd.read_csv("healthcare_dataset.csv", encoding='latin-1')

df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                55500 non-null  object 
 1   Age                 55500 non-null  int64  
 2   Gender              55500 non-null  object 
 3   Blood Type          55500 non-null  object 
 4   Medical Condition   55500 non-null  object 
 5   Date of Admission   55500 non-null  object 
 6   Doctor              55500 non-null  object 
 7   Hospital            55500 non-null  object 
 8   Insurance Provider  55500 non-null  object 
 9   Billing Amount      55500 non-null  float64
 10  Room Number         55500 non-null  int64  
 11  Admission Type      55500 non-null  object 
 12  Discharge Date      55500 non-null  object 
 13  Medication          55500 non-null  object 
 14  Test Results        55500 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 6.4

In [4]:
df.nunique()

,0
Name,49992
Age,77
Gender,2
Blood Type,8
Medical Condition,6
Date of Admission,1827
Doctor,40341
Hospital,39876
Insurance Provider,5
Billing Amount,50000


In [5]:
df.describe()

,Age,Billing Amount,Room Number
count,55500.000000,55500.000000,55500.000000
mean,51.539459,25539.316097,301.134829
std,19.602454,14211.454431,115.243069
min,13.000000,-2008.492140,101.000000
25%,35.000000,13241.224652,202.000000
50%,52.000000,25538.069376,302.000000
75%,68.000000,37820.508436,401.000000
max,89.000000,52764.276736,500.000000


In [6]:
df.isnull().sum()

,0
Name,0
Age,0
Gender,0
Blood Type,0
Medical Condition,0
Date of Admission,0
Doctor,0
Hospital,0
Insurance Provider,0
Billing Amount,0


In [7]:
df.duplicated().sum()

np.int64(534)

## **Data Cleaning: Standardization and Preprocessing**

In [8]:
# 1. Standardize column names
df.columns = df.columns.str.lower().str.replace(" ", "_")

# 2. Title-case the name field
df['name'] = df['name'].str.title()

# 3. Standardize categorical text fields
categorical_columns = ['gender', 'blood_type', 'medical_condition', 'doctor',
                       'hospital', 'insurance_provider', 'admission_type',
                       'medication', 'test_results']
for col in categorical_columns:
    df[col] = df[col].str.strip().str.title()

# 4. Convert date columns to datetime
df['date_of_admission'] = pd.to_datetime(df['date_of_admission'], errors='coerce')
df['discharge_date'] = pd.to_datetime(df['discharge_date'], errors='coerce')

# 5. Remove duplicates
df = df.drop_duplicates()

In [9]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 54966 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   name                54966 non-null  object        
 1   age                 54966 non-null  int64         
 2   gender              54966 non-null  object        
 3   blood_type          54966 non-null  object        
 4   medical_condition   54966 non-null  object        
 5   date_of_admission   54966 non-null  datetime64[ns]
 6   doctor              54966 non-null  object        
 7   hospital            54966 non-null  object        
 8   insurance_provider  54966 non-null  object        
 9   billing_amount      54966 non-null  float64       
 10  room_number         54966 non-null  int64         
 11  admission_type      54966 non-null  object        
 12  discharge_date      54966 non-null  datetime64[ns]
 13  medication          54966 non-null  object        


,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results
0,Bobby Jackson,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons And Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,Leslie Terry,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,Danny Smith,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook Plc,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,Andrew Watts,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers And Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,Adrienne Bell,43,Female,Ab+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [10]:
df['billing_amount'] = df['billing_amount'].round(2)


In [ ]:
df.to_csv('healthcare_dataset_cleaned.csv', index=False)
files.download('healthcare_dataset_cleaned.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Descriptive Statistics for Hypothesis Testing**

Hypothesis 1: Compares billing amounts between male and female patients

Hypothesis 2: Examines the relationship between patient age and visit frequency

Hypothesis 3: Compares billing amounts between patients with and without chronic conditions


In [14]:
def calculate_measures(data, name):
    print(f"\nMeasures for {name}:")
    print(f"  Mean: {np.mean(data):.2f}")
    print(f"  Median: {np.median(data):.2f}")
    print(f"  Standard Deviation: {np.std(data):.2f}")

# Hypothesis 1: Billing vs. Gender
calculate_measures(male_bills, "Billing Amount (Male)")
calculate_measures(female_bills, "Billing Amount (Female)")

# Hypothesis 2: Age vs. Visit Frequency
calculate_measures(visit_df['age'], "Age")
calculate_measures(visit_df['visit_count'], "Visit Count")

# Hypothesis 3: Billing vs. Chronic Conditions
if len(chronic_bills) > 1:
    calculate_measures(chronic_bills, "Billing Amount (Chronic Conditions)")
if len(non_chronic_bills) > 1:
    calculate_measures(non_chronic_bills, "Billing Amount (Non-Chronic Conditions)")



Measures for Billing Amount (Male):
  Mean: 25614.45
  Median: 25686.88
  Standard Deviation: 14214.92

Measures for Billing Amount (Female):
  Mean: 25474.10
  Median: 25427.85
  Standard Deviation: 14201.28

Measures for Age:
  Mean: 51.54
  Median: 52.00
  Standard Deviation: 19.61

Measures for Visit Count:
  Mean: 1.01
  Median: 1.00
  Standard Deviation: 0.08

Measures for Billing Amount (Non-Chronic Conditions):
  Mean: 25544.31
  Median: 25542.75
  Standard Deviation: 14208.28


In [15]:
print("\nSummary statistics:\n",df.describe())


Summary statistics:
                 age              date_of_admission  billing_amount  \
count  54966.000000                          54966    54966.000000   
mean      51.535185  2021-11-01 17:35:29.505512448    25544.306260   
min       13.000000            2019-05-08 00:00:00    -2008.490000   
25%       35.000000            2020-07-28 00:00:00    13243.722500   
50%       52.000000            2021-11-02 00:00:00    25542.750000   
75%       68.000000            2023-02-03 00:00:00    37819.857500   
max       89.000000            2024-05-07 00:00:00    52764.280000   
std       19.605661                            NaN    14208.409715   

        room_number                 discharge_date  
count  54966.000000                          54966  
mean     301.124404  2021-11-17 05:34:28.202161408  
min      101.000000            2019-05-09 00:00:00  
25%      202.000000            2020-08-13 00:00:00  
50%      302.000000            2021-11-18 00:00:00  
75%      401.000000          

# **Descriptive Statistics for All Numerical Columns**


In [24]:
print("\nDescriptive Statistics for Numerical Columns:")
for col in df.select_dtypes(include=np.number).columns:
  print(f"\n--- {col} ---")
  print(f"Mean: {df[col].mean():.2f}")
  print(f"Median: {df[col].median():.2f}")
  try:
    # Mode can return multiple values if they have the same highest frequency
    modes = df[col].mode()
    print(f"Mode: {', '.join([str(mode) for mode in modes])}")
  except Exception as e:
    print(f"Mode: Could not compute ({e})")
  print(f"Standard Deviation: {df[col].std():.2f}")


Descriptive Statistics for Numerical Columns:

--- age ---
Mean: 51.54
Median: 52.00
Mode: 38
Standard Deviation: 19.61

--- billing_amount ---
Mean: 25544.31
Median: 25542.75
Mode: 16208.95
Standard Deviation: 14208.41

--- room_number ---
Mean: 301.12
Median: 302.00
Mode: 393
Standard Deviation: 115.22

--- gender_male ---
Mean: 0.50
Median: 1.00
Mode: 1
Standard Deviation: 0.50

--- chronic_condition ---
Mean: 0.00
Median: 0.00
Mode: 0
Standard Deviation: 0.00
